<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/DEEPSEEK_AAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langgraph -q
!pip install langchain-google-genai -q
!pip install google-cloud-aiplatform --upgrade -q
!pip install python-dateutil
!pip install langchain -q

!pip install langchain-openai -q

import langchain

print(langchain.__version__)

## CASE 1

In [3]:
from openai import OpenAI
from google.colab import userdata
from typing import Optional, List

# Get API key
api_key = userdata.get('DEEPSEEK_API_KEY')

# Initialize the DeepSeek client
client = OpenAI(api_key=api_key, base_url="https://api.deepseek.com")

def find_flights(departure_city: str, arrival_city: str, departure_date: str, return_date: Optional[str] = None):
    # This is a placeholder function, as the DeepSeek reference does not show how to implement tool calling.
    return f"Simulated flight results for {departure_city} to {arrival_city}"

# NOTE: Tool use functionality is not demonstrated in the provided DeepSeek reference code.
# Therefore, the complex tool-calling logic from the original Gemini code cannot be replicated.

# Simple prompt-response without tool use
prompt_text = "Find me flights from New York to Los Angeles on 2024-12-24."

response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=[
        {"role": "system", "content": "You are a helpful travel agent."},
        {"role": "user", "content": prompt_text}
    ]
)

final_answer = response.choices[0].message.content
print(final_answer)

Of course! To find you the best flight options from New York to Los Angeles on **Tuesday, December 24, 2024**, I'll need a few more details to tailor the search to your preferences.

Could you please let me know:

1.  **Which New York airport do you prefer?**
    *   **JFK** (John F. Kennedy International)
    *   **LGA** (LaGuardia)
    *   **EWR** (Newark Liberty International)

2.  **Which Los Angeles airport do you prefer?**
    *   **LAX** (Los Angeles International) - Primary airport, most options.
    *   **BUR** (Hollywood Burbank) - Often more convenient for the Valley/Hollywood.
    *   **SNA** (John Wayne Orange County) - Good for Disneyland/Orange County.
    *   **ONT** (Ontario International) - Good for the Inland Empire.

3.  **What is your budget?**
4.  **Do you have a preferred airline, or are you looking for the cheapest/best value option?**
5.  **What is your approximate departure time preference?** (Morning, Afternoon, Evening)

Once you provide these details, I can

## CASE 2

In [5]:
from typing import Optional, List
import os
import json
from google.colab import userdata # For fetching API keys securely
from openai import OpenAI # DeepSeek API is compatible with OpenAI's client

# 1. Define the local function that the LLM can "call"
def display_cities(cities: list[str], preferences: Optional[str] = None):
    """Provides a list of cities based on the user's search query and preferences.

    Args:
        preferences (str): The user's preferences for the search, like skiing,
            beach, restaurants, bbq, etc.
        cities (list[str]): The list of cities being recommended to the user.

    Returns:
        list[str]: The list of cities being recommended to the user.
    """
    print(f"DEBUG: display_cities called with cities={cities}, preferences={preferences}")
    return cities

# 2. Define the tool declaration in OpenAI-compatible format
# This describes the function to the LLM.
display_cities_tool_declaration = {
    "type": "function",
    "function": {
        "name": "display_cities",
        "description": "Provides a list of cities based on the user's search query and preferences. Use this tool when the user asks for city recommendations.",
        "parameters": {
            "type": "object",
            "properties": {
                "cities": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": "List of cities being recommended to the user.",
                },
                "preferences": {
                    "type": "string",
                    "description": "The user's preferences for the search, like skiing, beach, restaurants, bbq, etc.",
                },
            },
            "required": ["cities"], # 'cities' is required for the tool to be called
        },
    }
}

# 3. Configure the DeepSeek client
try:
    # Fetch the DeepSeek API key from Colab secrets.
    # Ensure you have a secret named 'DEEPSEEK_API_KEY' set up.
    DEEPSEEK_API_KEY = userdata.get('DEEPSEEK_API_KEY')
except userdata.SecretNotFoundError:
    print("Error: 'DEEPSEEK_API_KEY' not found in Colab secrets. Please set it up.")
    exit()

# Initialize the OpenAI client, pointing it to the DeepSeek API endpoint.
client = OpenAI(api_key=DEEPSEEK_API_KEY, base_url="https://api.deepseek.com/v1")

# 4. Define the user's prompt
# FIX: Make the prompt highly specific to provide all necessary information for the tool call.
user_prompt = "I'd like to take a family ski trip in North America. We're looking for beginner-friendly slopes, a ski school, and a resort with a charming village atmosphere and family-friendly dining options."

# 5. Send the request to the DeepSeek model with the tool declaration
# FIX: Explicitly set 'tool_choice' to force the model to call the 'display_cities' tool.
try:
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "user", "content": user_prompt}
        ],
        tools=[display_cities_tool_declaration],
        tool_choice={"type": "function", "function": {"name": "display_cities"}} # Force the tool call
    )

    # 6. Check for a tool call in the response
    if response.choices[0].message.tool_calls:
        function_call = response.choices[0].message.tool_calls[0].function
        print(f"Function to call: {function_call.name}")

        # Arguments are returned as a JSON string, so they need to be parsed
        args = json.loads(function_call.arguments)
        print(f"Arguments: {args}")

        # In a real application, you would now execute your local function:
        # result = display_cities(**args)
        # print(f"Tool execution result: {result}")
    else:
        print("No function call found in the response.")
        print(f"Model's direct response: {response.choices[0].message.content}")

except Exception as e:
    print(f"An error occurred: {e}")


Function to call: display_cities
Arguments: {'cities': ['Whistler, British Columbia', 'Park City, Utah', 'Breckenridge, Colorado', 'Lake Placid, New York', 'Stowe, Vermont', 'Banff, Alberta', 'Aspen, Colorado', 'Telluride, Colorado', 'Sun Valley, Idaho', 'Tremblant, Quebec'], 'preferences': 'skiing, beginner-friendly slopes, ski school, charming village atmosphere, family-friendly dining options, North America'}


## CASE 3

In [16]:
import os
import json
from openai import OpenAI
from google.colab import userdata
from typing import Optional, List, Dict, Any

# 1. Define the Tool Declaration
find_flights_tool_declaration = {
    "type": "function",
    "function": {
        "name": "find_flights",
        "description": "Finds available flights based on the specified criteria. Use this tool only when the user's query explicitly asks for flight information.",
        "parameters": {
            "type": "object",
            "properties": {
                "departure_city": {
                    "type": "string",
                    "description": "City of departure"
                },
                "arrival_city": {
                    "type": "string",
                    "description": "City of arrival"
                },
                "departure_date": {
                    "type": "string",
                    "description": "Departure date (YYYY-MM-DD)"
                },
                "return_date": {
                    "type": "string",
                    "description": "Return date (YYYY-MM-DD), if any"
                },
                "cabin_class": {
                    "type": "string",
                    "description": "cabin class, e.g., economy, business, first"
                },
                "num_passengers": {
                    "type": "integer",
                    "description": "number of passengers"
                }
            },
            "required": ["departure_city", "arrival_city", "departure_date"]
        }
    }
}

# 2. Define the Agent's State and Logic
def run_flight_agent(user_query: str):
    """
    Manually runs a simple agentic loop to find flights using the DeepSeek API.
    """
    # Get API key from Colab secrets.
    try:
        api_key = userdata.get('DEEPSEEK_API_KEY')
    except userdata.SecretNotFoundError:
        return "API key not found. Please create a secret named 'DEEPSEEK_API_KEY' in Google Colab."

    # Initialize the DeepSeek client using the OpenAI-compatible base_url.
    client = OpenAI(api_key=api_key, base_url="https://api.deepseek.com/v1")

    # The messages list represents the chat history.
    messages = [{"role": "user", "content": user_query}]

    # The while loop acts as the agent's graph.
    while True:
        try:
            # Call the DeepSeek API with the user's query and the tool declaration.
            response = client.chat.completions.create(
                model="deepseek-chat",
                messages=messages,
                tools=[find_flights_tool_declaration],
                # Force the function call to bypass the model's information-gathering loop
                tool_choice={"type": "function", "function": {"name": "find_flights"}}
            )

            # Check if the model has decided to make a tool call.
            if response.choices[0].message.tool_calls:
                tool_call = response.choices[0].message.tool_calls[0].function

                print(f"Agent's final decision: Call the '{tool_call.name}' tool.")
                print(f"Arguments: {tool_call.arguments}")

                return f"Simulated tool call with arguments: {tool_call.arguments}"

            else:
                model_response = response.choices[0].message.content
                print(f"Agent's response: {model_response}")

                return f"Agent's response: {model_response}"

        except Exception as e:
            return f"An error occurred: {e}"


# 3. Run the Agent
user_input = "I need to book a flight from New York to San Francisco for 2 passengers in business class on 2025-10-26."
agent_result = run_flight_agent(user_input)
print(f"\nResult of the agent run: {agent_result}")

Agent's final decision: Call the 'find_flights' tool.
Arguments: {"departure_city": "New York", "arrival_city": "San Francisco", "departure_date": "2025-10-26", "num_passengers": 2, "cabin_class": "business"}

Result of the agent run: Simulated tool call with arguments: {"departure_city": "New York", "arrival_city": "San Francisco", "departure_date": "2025-10-26", "num_passengers": 2, "cabin_class": "business"}


## CASE4

In [1]:
from typing import Optional, List, Dict, Any
import os
import json
from google.colab import userdata
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field # Correct Pydantic Import
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from warnings import filterwarnings
filterwarnings('ignore')

# 1. Define Tools
@tool
def find_flights(
    departure_city: str,
    arrival_city: str,
    departure_date: str,
    return_date: Optional[str] = None,
    cabin_class: Optional[str] = "economy",
    num_passengers: int = 1,
):
    """
    Finds available flights based on the specified criteria.
    """
    print(f"DEBUG: find_flights called with args: {locals()}")
    return f"Simulated flight results for {departure_city} to {arrival_city} on {departure_date}. Passengers: {num_passengers}, Cabin: {cabin_class}"

# 2. Define the Agent's State
class AgentState(BaseModel):
    user_query: str
    chat_history: List[tuple[str, str]] = Field(default_factory=list)
    flight_results: Optional[Any] = None
    flight_info: Dict[str, Any] = Field(default_factory=dict)
    intermediate_response: Optional[str] = None
    response: Optional[str] = None

# 3. Define the Agent's Chains
def create_agent(tools: list):
    api_key = userdata.get('DEEPSEEK_API_KEY')
    llm = ChatOpenAI(
        model="deepseek-chat",
        api_key=api_key,
        base_url="https://api.deepseek.com/v1"
    )
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a helpful AI travel agent with access to flight search tools."),
            ("human", "{user_query}"),
            ("placeholder", "{chat_history}"),
        ]
    )
    tool_chain = prompt | llm.bind_tools(tools)
    return {"tool_chain": tool_chain}

def agent_node(state: AgentState, config: Optional[Dict[str, Any]] = None):
    print(f"DEBUG: Entering agent_node. State: {state}")
    agent = create_agent(tools=[find_flights])

    if state.flight_results is not None:
        print("DEBUG: flight_results is not None, exiting agent_node.")
        return state

    print("DEBUG: Invoking tool_chain.")
    result = agent["tool_chain"].invoke({
        "user_query": state.user_query,
        "chat_history": state.chat_history
    })

    tool_calls = result.additional_kwargs.get("tool_calls")
    if tool_calls:
        print(f"DEBUG: Tool calls found. Returning tool call to graph: {tool_calls}")
        tool_call = tool_calls[0]
        parsed_args = json.loads(tool_call['function']['arguments'])
        return {
            "flight_info": parsed_args,
            "flight_results": f"Simulated flight results for {parsed_args}",
        }
    else:
        print("DEBUG: No tool calls found. Returning AI message.")
        return {"intermediate_response": result.content}

def should_continue(state: AgentState):
    if state.flight_results:
        return "exit"
    else:
        return "continue"

# Create the graph
graph_builder = StateGraph(AgentState)
graph_builder.add_node("agent_node", agent_node)
graph_builder.set_entry_point("agent_node")
graph_builder.add_conditional_edges("agent_node", should_continue, {"continue": "agent_node", "exit": END})
graph = graph_builder.compile()

# 5. Run the Agent
user_query_initial = "Find me flights from New York to Los Angeles on 2024-12-20 for 2 people in business class."
inputs1 = {"user_query": user_query_initial, "chat_history": []}
print("Running initial query...")
final_state1 = graph.invoke(inputs1, {"recursion_limit": 50})
print("\nState after initial query:", final_state1)

Running initial query...
DEBUG: Entering agent_node. State: user_query='Find me flights from New York to Los Angeles on 2024-12-20 for 2 people in business class.' chat_history=[] flight_results=None flight_info={} intermediate_response=None response=None
DEBUG: Invoking tool_chain.
DEBUG: Tool calls found. Returning tool call to graph: [{'id': 'call_0_6db97e07-4e7c-4d10-826e-240f88ecf2f9', 'function': {'arguments': '{"departure_city": "New York", "arrival_city": "Los Angeles", "departure_date": "2024-12-20", "cabin_class": "business", "num_passengers": 2}', 'name': 'find_flights'}, 'type': 'function', 'index': 0}]

State after initial query: {'user_query': 'Find me flights from New York to Los Angeles on 2024-12-20 for 2 people in business class.', 'chat_history': [], 'flight_results': "Simulated flight results for {'departure_city': 'New York', 'arrival_city': 'Los Angeles', 'departure_date': '2024-12-20', 'cabin_class': 'business', 'num_passengers': 2}", 'flight_info': {'departure_c

## CASE5

In [2]:
from typing import Optional, List, Dict, Any
import os
import json
from google.colab import userdata
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from warnings import filterwarnings
filterwarnings('ignore')

# 1. Define Tools
@tool
def find_flights(
    departure_city: str,
    arrival_city: str,
    departure_date: str,
    return_date: Optional[str] = None,
    cabin_class: Optional[str] = "economy",
    num_passengers: int = 1,
):
    """
    Finds available flights based on the specified criteria.
    """
    print(f"DEBUG: find_flights called with args: {locals()}")
    return f"Simulated flight results for {departure_city} to {arrival_city} on {departure_date}. Passengers: {num_passengers}, Cabin: {cabin_class}"


# 2. Define the Agent's State
class AgentState(BaseModel):
    user_query: str
    chat_history: List[tuple[str, str]] = Field(default_factory=list)
    flight_results: Optional[Any] = None
    flight_info: Dict[str, Any] = Field(default_factory=dict)
    intermediate_response: Optional[str] = None
    response: Optional[str] = None


# 3. Define the Agent's Chains
def create_agent(tools: list):
    api_key = userdata.get('DEEPSEEK_API_KEY')
    llm = ChatOpenAI(
        model="deepseek-reasoner", # THE ONLY CHANGE: Use the reasoner model
        api_key=api_key,
        base_url="https://api.deepseek.com/v1"
    )
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a helpful AI travel agent with access to flight search tools."),
            ("human", "{user_query}"),
            ("placeholder", "{chat_history}"),
        ]
    )
    tool_chain = prompt | llm.bind_tools(tools)
    return {"tool_chain": tool_chain}


def agent_node(state: AgentState, config: Optional[Dict[str, Any]] = None):
    print(f"DEBUG: Entering agent_node. State: {state}")
    agent = create_agent(tools=[find_flights])

    if state.flight_results is not None:
        print("DEBUG: flight_results is not None, exiting agent_node.")
        return state

    print("DEBUG: Invoking tool_chain.")
    result = agent["tool_chain"].invoke({
        "user_query": state.user_query,
        "chat_history": state.chat_history
    })

    tool_calls = result.additional_kwargs.get("tool_calls")
    if tool_calls:
        print(f"DEBUG: Tool calls found. Returning tool call to graph: {tool_calls}")
        tool_call = tool_calls[0]
        parsed_args = json.loads(tool_call['function']['arguments'])
        return {
            "flight_info": parsed_args,
            "flight_results": f"Simulated flight results for {parsed_args}",
        }
    else:
        print("DEBUG: No tool calls found. Returning AI message.")
        return {"intermediate_response": result.content}


def should_continue(state: AgentState):
    if state.flight_results:
        return "exit"
    else:
        return "continue"


# Create the graph
graph_builder = StateGraph(AgentState)
graph_builder.add_node("agent_node", agent_node)
graph_builder.set_entry_point("agent_node")
graph_builder.add_conditional_edges("agent_node", should_continue, {"continue": "agent_node", "exit": END})
graph = graph_builder.compile()


# 5. Run the Agent
user_query_initial = "Find me flights from New York to Los Angeles on 2024-12-20 for 2 people in business class."
inputs1 = {"user_query": user_query_initial, "chat_history": []}
print("Running initial query...")
final_state1 = graph.invoke(inputs1, {"recursion_limit": 50})
print("\nState after initial query:", final_state1)

Running initial query...
DEBUG: Entering agent_node. State: user_query='Find me flights from New York to Los Angeles on 2024-12-20 for 2 people in business class.' chat_history=[] flight_results=None flight_info={} intermediate_response=None response=None
DEBUG: Invoking tool_chain.
DEBUG: Tool calls found. Returning tool call to graph: [{'id': 'call_0_6a84f30a-0c17-4a43-979f-429532ebd98c', 'function': {'arguments': '{"departure_city": "New York", "arrival_city": "Los Angeles", "departure_date": "2024-12-20", "cabin_class": "business", "num_passengers": 2}', 'name': 'find_flights'}, 'type': 'function', 'index': 0}]

State after initial query: {'user_query': 'Find me flights from New York to Los Angeles on 2024-12-20 for 2 people in business class.', 'chat_history': [], 'flight_results': "Simulated flight results for {'departure_city': 'New York', 'arrival_city': 'Los Angeles', 'departure_date': '2024-12-20', 'cabin_class': 'business', 'num_passengers': 2}", 'flight_info': {'departure_c